In [ ]:
# %% [markdown]
# # Movie reviews Classification

# %% [markdown]
# <img src="https://frenzy86.s3.eu-west-2.amazonaws.com/python/nlp/rev.png" width="1200">
# 
# 

# %%
import numpy as np
import pandas as pd
import re
from sklearn.datasets import load_files

import warnings
warnings.filterwarnings('ignore')

# %%
!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/nlp/txt_sentoken.zip

# %%
!unzip txt_sentoken.zip

# %%
movie_data = load_files(r"txt_sentoken")
X_, y = movie_data.data, movie_data.target

# %%
unique, counts = np.unique(y, return_counts=True)

print(np.asarray((unique, counts)).T)

# %%
X_

# %%
type(X_)

# %%
def pulizia_preliminare(lista_stringhe, lista_regex):
    stringhe_pulite = []
    for stringa in lista_stringhe:
        stringa = stringa.decode()  # Converte da byte a stringa
        for regex in lista_regex:
            stringa = re.sub(regex, '', stringa)
        stringhe_pulite.append(stringa)
    return stringhe_pulite

# %%
# Esempio di regex per rimuovere caratteri speciali
regex = [
    # r"[!@#$%^&*()]",
    # r"[\_\-\"\?\;\'\,\.\:\/\\\n]",
    # r"(?<!\S)\\d+(?!\S)"
    r"[^a-zA-Z\s]"
]

# Applica la pulizia preliminare
stringhe_pulite = pulizia_preliminare(X_, regex)

# %%
stringhe_pulite

# %%
def rimuovi_spazi_in_eccesso(lista_stringhe):
    stringhe_pulite = []
    for stringa in lista_stringhe:
        stringa_pulita = ' '.join(stringa.split())
        stringhe_pulite.append(stringa_pulita)
    return stringhe_pulite

# %%
stringhe_pulite = rimuovi_spazi_in_eccesso(stringhe_pulite)

# %%
stringhe_pulite

# %%
res = " ".join(stringhe_pulite)

# %%
print(type(res))


